In [1]:
%load_ext autoreload

In [2]:
%autoreload 2
from FEMOL import RectangleQuadMesh, FEM_Problem
from FEMOL.utils import *
from FEMOL.materials import general_isotropic
import FEMOL.materials
import numpy as np
import time
import scipy.linalg
np.set_printoptions(linewidth=110)
%config Completer.use_jedi = False

In [3]:
# assignment 2
material = FEMOL.materials.flax_epoxy()
plies_s = [0, 30, -30, 45, -45, 90]
plies_t = np.hstack([plies_s, np.flip(plies_s)])
layup = FEMOL.laminate.Layup(material=material, plies=plies_s, symetric=True)

In [4]:
# assignment 3
material = FEMOL.materials.T300_N5208()
plies_s = [0, 45, -45, 90, 90, 30, -30, 90]
plies_t = np.hstack([plies_s, np.flip(plies_s)])
layup = FEMOL.laminate.Layup(material=material, plies=plies_t, symetric=False)

In [5]:
# assignment 4
reference_D_mat = np.array([[ 6292.09,   350.25,    15.41],
                            [  350.25,  1599.72,     2.67],
                            [   15.41,    2.67,    535.73], ])# kN * mm

reference_A_mat = np.array([[211.678,   12.489,    0.],
                            [12.489,     60.27,    0.],
                            [0.,            0.,   18.899], ])

material = FEMOL.materials.T300_N5208()

plies_s = [0, 0, 0, 20, -20, 90]  # s
plies_t = np.hstack([plies_s, np.flip(plies_s)])

layup = FEMOL.laminate.Layup(material=material, plies=plies_t, symetric=False, z_core=0, h_core=0.010)

np.allclose(layup.D_mat, reference_D_mat, 10e-3)

True

In [6]:
# Test for zc
material = FEMOL.materials.general_carbon()

# 1. summing D matrices (OK)

plies1 = [0, 90, -45, 45]  # s
layup1 = FEMOL.laminate.Layup(material=material, plies=plies1, symetric=True)  # no core
plies2 = [45, -45]  # s
layup2 = FEMOL.laminate.Layup(material=material, plies=plies2, symetric=True, h_core=layup1.h)
plies3 = [45, -45, 0, 90, -45, 45]  # s
layup3 = FEMOL.laminate.Layup(material=material, plies=plies3, symetric=True)  # no core
combined_D = layup1.D_mat + layup2.D_mat
np.allclose(combined_D, layup3.D_mat)


True

In [7]:
# 2. Summing two offset layups (OK)

# total layup
plies_all = [45, -45, -45, 45]
layup_all = FEMOL.laminate.Layup(material=material, plies=plies_all, symetric=False, h_core=0.010)  # 10mm core

# two halves
plies_bottom = [45, -45]
layup_b = FEMOL.laminate.Layup(material=material, plies=plies_bottom, symetric=False, h_core=0, z_core= -0.005 - material.hi) 
plies_top = [-45, 45]
layup_t = FEMOL.laminate.Layup(material=material, plies=plies_top, symetric=False, h_core=0, z_core= 0.005 + material.hi) 

np.allclose(layup_b.D_mat, layup_t.D_mat)
np.allclose(layup_b.D_mat + layup_t.D_mat, layup_all.D_mat)

True